In [187]:
import pandas as pd 
import os 
from dotenv import load_dotenv
import requests 
import json 
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
import numpy as np

In [188]:
load_dotenv()

True

In [189]:
api_key = os.getenv('BLOCKSPAN_API_KEY')

In [190]:
url = "https://api.blockspan.com/v1/exchanges/collectionsranking?chain=eth-main&exchange=opensea&ranking=seven_day_volume&page_size=25"
headers = {
    "accept": "application/json",
    "X-API-KEY": "PwIPZGmaJpLNse6wldVqlrvUIJgg7suk"
}

In [191]:
response = requests.get(url, headers=headers).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "chain": "eth-main",
    "cursor": "MzkwLjE1NTA0MTc5OTgwMDM2",
    "exchange": "opensea",
    "per_page": "25",
    "ranking": "seven_day_volume",
    "results": [
        {
            "contracts": [
                {
                    "contract_address": "0xc178994cb9b66307cd62db8b411759dd36d9c2ee"
                }
            ],
            "description": "The Legend of COCKPUNCH\u2122 is the tale of a fantastical realm, a universe of the bizarre from the mind of bestselling author [Tim Ferriss](https://tim.blog/about). Artwork and stories are the gateway drug in this Emergent Long Fiction (ELF) project. Learn more at [cockpunch.com](https://cockpunch.com). Follow [@cockpunch](https://twitter.com/cockpunch) and [@tferriss](https://twitter.com/tferriss) on Twitter.\r\nBefore buying this NFT, please read the NFT License Agreement at [tim.blog/cockpunch-license](https://tim.blog/cockpunch-license). This explains your rights and restrictions.",
            "key": "cockpunch",
 

In [217]:
nft_df = pd.DataFrame(response['results'])
nft_df = nft_df.drop(['description'], ['update at'],['contracts'])

/Users/alexandrapaiz/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


ValueError: Cannot specify both 'labels' and 'index'/'columns'

In [218]:


nft_df.head()



,key,name,description,update_at,total_volume,one_day_volume,seven_day_volume,thirty_day_volume,total_sales,one_day_sales,seven_day_sales,thirty_day_sales,total_average_price,one_day_average_price,seven_day_average_price,thirty_day_average_price,contracts
0,cockpunch,The Legend of CØCKPUNCH™,The Legend of COCKPUNCH™ is the tale of a fant...,2022-12-13T05:32:35.984Z,4724.8470141073485,94.82747490000001,4724.847014107358,4724.847014107358,3894,131,3894,3894,1.2133659512345527,0.7238738541984734,1.2133659512345552,1.2133659512345552,[{'contract_address': '0xc178994cb9b66307cd62d...
1,valhalla,Valhalla,Valhalla is a crypto native brand for gamers. ...,2022-12-13T05:42:44.955Z,8484.929966982996,809.5112387899013,4509.194574951207,8484.92996698298,10448,620,4941,10448,0.8121104486009759,1.3056632883708086,0.9126076856812805,0.8121104486009744,[{'contract_address': '0x231d3559aa848bf10366f...
2,art-blocks,Art Blocks,Art Blocks is dedicated to bringing compelling...,2022-12-07T23:34:39.755Z,461032.1315977061,370.18108999999987,2757.0796924600677,8762.908062164192,205885,334,1726,6380,2.239270134287132,1.1083266167664667,1.597381050092739,1.3734965614677417,[{'contract_address': '0xa7d8d9ef8d8ce8992df33...
3,cryptopunks,CryptoPunks,"CryptoPunks launched as a fixed set of 10,000 ...",2022-12-13T04:10:11.293Z,1074113.0570093382,196.91000000000003,2576.270000000001,13778.810000000007,22037,4,40,205,48.741346690082054,49.227500000000006,64.40675000000002,67.2137073170732,[{'contract_address': '0xb47e3cd837ddf8e4c57f0...
4,mutant-ape-yacht-club,Mutant Ape Yacht Club,The MUTANT APE YACHT CLUB is a collection of u...,2022-12-13T05:32:36.424Z,471756.2364785258,352.46842,2147.8395828333337,8675.78246512353,36097,22,140,635,13.069125868590902,16.02129181818182,15.341711305952384,13.662649551375637,[{'contract_address': '0x60e4d786628fea6478f78...


In [220]:
nft_df.to_csv('Nftdata.csv')

In [216]:

#nft_df['Change'] = nft_df['thirty_day_average_price'].subtract(nft_df['one_day_average_price'])


nft_df['Challenge'] = float(nft_df['thirty_day_average_price']) - float(nft_df['one_day_average_price'])


TypeError: cannot convert the series to <class 'float'>